In [2]:
import os
import glob
labels_path = glob.glob('/kaggle/input/label/label/*.xml')
len(labels_path)
! pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.5/749.5 kB 28.8 MB/s eta 0:00:00


In [42]:
images_path_all = glob.glob('/kaggle/input/images/images/*/*.jpg')
len(images_path_all)

2306

In [4]:
len(set([os.path.basename(x) for x in images_path_all]))

2294

In [5]:
from collections import Counter
counter = Counter([os.path.basename(x) for x in images_path_all])
duplicates = [x.split(".")[0] for x,y in counter.items() if y > 1]
duplicates

['img_03_436068500_00002',
 'img_07_425503000_00061',
 'img_01_425503100_00018',
 'img_02_425392000_00984',
 'img_06_3436814000_00687',
 'img_07_436163600_01161',
 'img_06_425505500_00052',
 'img_04_425503600_00017',
 'img_03_425506300_00018',
 'img_07_425502900_00052',
 'img_07_425391800_00054',
 'img_06_425502900_00052']

In [6]:
for label_path in labels_path:
    for duplicate in duplicates:
        if duplicate in label_path:
            print(label_path)
# [x for x in labels_path if x in ]

In [43]:
images_path_1 = glob.glob('/kaggle/input/images/images/silk_spot/*.jpg')
print(len(images_path_1))

labels_path_1 = [path for path in labels_path for img in images_path_1 if path.split('/')[-1].split('.xml')[0] in img]
print(len(labels_path_1))

xmls_data_1 = [p.split('/')[-1].split('.')[0] for p in labels_path_1]
images_data_1 = [img for img in images_path_1 if (img.split('/')[-1].split)('.jpg')[0] in xmls_data_1]
print(len(images_data_1),len(xmls_data_1))

labels_path_1.sort(key=lambda x: x.split("/")[-1].split('.xml')[0])
images_data_1.sort(key=lambda x: x.split("/")[-1].split('.jpg')[0])
labels_path_1[:9],images_data_1[:9]

650
649
649 649


(['/kaggle/input/label/label/img_01_3436814600_00259.xml',
  '/kaggle/input/label/label/img_01_3436815300_00473.xml',
  '/kaggle/input/label/label/img_01_3436815300_00477.xml',
  '/kaggle/input/label/label/img_01_3436815300_00478.xml',
  '/kaggle/input/label/label/img_01_3437011300_00918.xml',
  '/kaggle/input/label/label/img_01_425005700_00154.xml',
  '/kaggle/input/label/label/img_01_425005700_00156.xml',
  '/kaggle/input/label/label/img_01_425005700_00158.xml',
  '/kaggle/input/label/label/img_01_425005700_00159.xml'],
 ['/kaggle/input/images/images/silk_spot/img_01_3436814600_00259.jpg',
  '/kaggle/input/images/images/silk_spot/img_01_3436815300_00473.jpg',
  '/kaggle/input/images/images/silk_spot/img_01_3436815300_00477.jpg',
  '/kaggle/input/images/images/silk_spot/img_01_3436815300_00478.jpg',
  '/kaggle/input/images/images/silk_spot/img_01_3437011300_00918.jpg',
  '/kaggle/input/images/images/silk_spot/img_01_425005700_00154.jpg',
  '/kaggle/input/images/images/silk_spot/img_01

In [44]:
os.makedirs('/kaggle/working/for_data', exist_ok=True)

In [54]:
import xml.etree.ElementTree as ET
import glob
import re
import torch

for label_path in labels_path_1:
    tree = ET.parse(label_path)
    #ET.dump(tree)
    root = tree.getroot()
    
    for object_element in root.findall(".//object"):
    
        bndbox_element = object_element.find("bndbox")

        xmin = int(bndbox_element.find("xmin").text)
        ymin = int(bndbox_element.find("ymin").text)
        xmax = int(bndbox_element.find("xmax").text)
        ymax = int(bndbox_element.find("ymax").text)

        width = (xmax - xmin)/2048
        height = (ymax - ymin)/1000

        xc = xmin/2048 + width/2
        yc = ymin/1000 + height/2

        name_element = object_element.find("name")
        
        match = re.search(r'\d+', name_element.text)
        if match:
            num_class = int(match.group())
            if num_class == 6:
                n = 1
            else:
                n = 0 
            #num_class = int(match.group())
            #print(num_class)
            
            file_name = label_path.split('/')[5].split('.xml')[0]
            #print(file_name)
        
            with open(os.path.join('/kaggle/working/for_data', f'{file_name}.txt'), 'a') as file:
                # Запись в файл
                file.write(f'{n} {xc} {yc} {width} {height}\n')
                #print(f'{num_class} {xc} {yc} {width} {height}\n')
        else:
            print("No digits found in name_element.text")
            print(label_path)



In [55]:
def count_txt_files(folder_path):
    txt_files_count = 0
    for file in os.listdir(folder_path):
        if file.endswith(".txt"):
            txt_files_count += 1
    return txt_files_count


folder_path = "/kaggle/working/for_data"
txt_files_count = count_txt_files(folder_path)
print(f"Количество файлов .txt в папке: {txt_files_count}")

Количество файлов .txt в папке: 649


In [10]:
ls /kaggle/input/images/images

 crease/         inclusion/   punching_hole/   silk_spot/        water_spot/
 crescent_gap/   oil_spot/    rolled_pit/     'waist folding'/   welding_line/


In [56]:
images_data_folder = '/kaggle/input/images/images'
images_path_0 = []
subfolders = [f.path for f in os.scandir(images_data_folder) if f.is_dir() and f.path != '/kaggle/input/images/images/silk_spot']
#print(len(subfolders))
mask = '/*.jpg'
for i in range(8):
    temp = subfolders[i] + mask
    if subfolders[i] == '/kaggle/input/images/images/crease' or subfolders[i] == '/kaggle/input/images/images/rolled_pit':
        images_path_0.extend(glob.glob(temp))
    else:
        images_path_0.extend(glob.glob(temp)[:81])

temp = subfolders[8] + mask
images_path_0.extend(glob.glob(temp)[:80])

len(images_path_0)
#print(images_path_1)

649

In [57]:
labels_path_0 = [path for path in labels_path for img in images_path_0 if path.split('/')[-1].split('.xml')[0] in img]
print(len(labels_path_1))

xmls_data_0 = [p.split('/')[-1].split('.')[0] for p in labels_path_0]
images_data_0 = [img for img in images_path_0 if (img.split('/')[-1].split)('.jpg')[0] in xmls_data_0]
print(len(images_data_0),len(xmls_data_0))

labels_path_0.sort(key=lambda x: x.split("/")[-1].split('.xml')[0])
images_data_0.sort(key=lambda x: x.split("/")[-1].split('.jpg')[0])
labels_path_0[:9],images_data_0[:9]

649
637 637


(['/kaggle/input/label/label/img_01_3436789500_00004.xml',
  '/kaggle/input/label/label/img_01_424799300_01133.xml',
  '/kaggle/input/label/label/img_01_424799600_00002.xml',
  '/kaggle/input/label/label/img_01_424826100_00001.xml',
  '/kaggle/input/label/label/img_01_425006100_00947.xml',
  '/kaggle/input/label/label/img_01_425006200_00373.xml',
  '/kaggle/input/label/label/img_01_425006200_01173.xml',
  '/kaggle/input/label/label/img_01_425006600_00001.xml',
  '/kaggle/input/label/label/img_01_425006600_00499.xml'],
 ['/kaggle/input/images/images/crease/img_01_3436789500_00004.jpg',
  '/kaggle/input/images/images/crescent_gap/img_01_424799300_01133.jpg',
  '/kaggle/input/images/images/crescent_gap/img_01_424799600_00002.jpg',
  '/kaggle/input/images/images/crescent_gap/img_01_424826100_00001.jpg',
  '/kaggle/input/images/images/crescent_gap/img_01_425006100_00947.jpg',
  '/kaggle/input/images/images/water_spot/img_01_425006200_00373.jpg',
  '/kaggle/input/images/images/crescent_gap/i

In [58]:
len(labels_path), len(images_path_0), len(labels_path_0)

(2280, 649, 637)

In [59]:
#count = 0
file_names = []

for label_path in labels_path_0:
    tree = ET.parse(label_path)
    #print(tree)
    #ET.dump(tree)
    root = tree.getroot()
    
    for object_element in root.findall(".//object"):
        #iner_count = 0
    
        bndbox_element = object_element.find("bndbox")
        
        if bndbox_element is None:
            print('bndbox_element is None')

        xmin_element = bndbox_element.find("xmin")
        ymin_element = bndbox_element.find("ymin")
        xmax_element = bndbox_element.find("xmax")
        ymax_element = bndbox_element.find("ymax")
        
        if xmin_element is None or ymin_element is None or xmax_element is None or ymax_element is None:
            print('xmin_element is None or ymin_element is None or xmax_element is None or ymax_element is None')
        
        xmin = int(xmin_element.text)
        ymin = int(ymin_element.text)
        xmax = int(xmax_element.text)
        ymax = int(ymax_element.text)
             
        width = (xmax - xmin)/2048
        height = (ymax - ymin)/1000

        xc = xmin/2048 + width/2
        yc = ymin/1000 + height/2
        
        if name_element is None:
            print('name_element is None')
            
        name_element = object_element.find("name")
        
        match = re.search(r'\d+', name_element.text)
        if match:
            num_class = int(match.group())
            if num_class == 6:
                n = 1
            else:
                n = 0  
            file_name = label_path.split('/')[5].split('.xml')[0]
        
            with open(os.path.join('/kaggle/working/for_data', f'{file_name}.txt'), 'a') as file:
#                 file.write(f'{n} {xc} {yc} {width} {height}\n')
#                 iner_count = 1
                try:
                    file.write(f'{n} {xc} {yc} {width} {height}\n')
                except Exception as e:
                    print(f"An error occurred while writing to file: {e}")
        else:
            print("No digits found in name_element.text")
            print(label_path)
#     count += iner_count
#     iner_count = 0
# print(count)

In [71]:
def count_txt_files(folder_path):
    txt_files_count = 0
    for file in os.listdir(folder_path):
        if file.endswith(".txt") or file.endswith(".jpg"):
            txt_files_count += 1
    return txt_files_count


folder_path = "/kaggle/working/train"
txt_files_count = count_txt_files(folder_path)
print(f"Количество файлов .txt в папке: {txt_files_count}")

Количество файлов .txt в папке: 1798


In [52]:
# import shutil
# def clear_folder(folder_path):
#     if os.path.exists(folder_path):
#         shutil.rmtree(folder_path)
#     os.makedirs(folder_path)
# clear_folder('/kaggle/working/for_data')

In [61]:
import random
from shutil import copyfile


source_folder = "/kaggle/working/for_data" 
train_folder = "/kaggle/working/train"
test_folder = "/kaggle/working/test"
valid_folder = "/kaggle/working/valid"

# subfolders = [f.path for f in os.scandir(images_data_folder) if f.is_dir()]
# print(subfolders)
#random.shuffle(subfolders)

for folder in [train_folder, test_folder, valid_folder]:
    os.makedirs(folder, exist_ok=True)


In [62]:
from pathlib import Path
from typing import Union

source_path = Path(source_folder)

txt_files_1 = []

for file in source_path.glob("*.txt"):
    for label in labels_path_1:
        if file.stem == Path(label).stem:
            txt_files_1.append(file)
print(txt_files_1[:10])

img_source_path = Path('/kaggle/input/images/images')
jpg_files_1 = []

for file in img_source_path.rglob("*.jpg"):
    for label in labels_path_1:
        if file.stem == Path(label).stem:
            jpg_files_1.append(file)
print(jpg_files_1[:10])

txt_files_0 = []

for file in source_path.glob("*.txt"):
    for label in labels_path_0:
        if file.stem == Path(label).stem:
            txt_files_0.append(file)
print(txt_files_0[:10])

jpg_files_0 = []

for file in img_source_path.rglob("*.jpg"):
    for label in labels_path_0:
        if file.stem == Path(label).stem:
            jpg_files_0.append(file)
print(jpg_files_0[:10])


[PosixPath('/kaggle/working/for_data/img_05_4406743300_00535.txt'), PosixPath('/kaggle/working/for_data/img_01_425005700_00444.txt'), PosixPath('/kaggle/working/for_data/img_07_4406645900_00687.txt'), PosixPath('/kaggle/working/for_data/img_06_4406743300_00096.txt'), PosixPath('/kaggle/working/for_data/img_03_436152900_00543.txt'), PosixPath('/kaggle/working/for_data/img_03_436152900_00605.txt'), PosixPath('/kaggle/working/for_data/img_03_425505400_01021.txt'), PosixPath('/kaggle/working/for_data/img_01_425005700_00196.txt'), PosixPath('/kaggle/working/for_data/img_06_4406743300_00042.txt'), PosixPath('/kaggle/working/for_data/img_03_436150300_00317.txt')]
[PosixPath('/kaggle/input/images/images/silk_spot/img_05_4406743300_00423.jpg'), PosixPath('/kaggle/input/images/images/silk_spot/img_01_425005700_00233.jpg'), PosixPath('/kaggle/input/images/images/silk_spot/img_01_425005700_00410.jpg'), PosixPath('/kaggle/input/images/images/silk_spot/img_02_SIS001522_00555.jpg'), PosixPath('/kaggl

In [63]:
txt_files_0 = sorted(txt_files_0, key=lambda x: x.stem)
jpg_files_0 = sorted(jpg_files_0, key=lambda x: x.stem)

for txt, jpg in zip(txt_files_0, jpg_files_0):
    assert txt.stem == jpg.stem
    
txt_files_1 = sorted(txt_files_1, key=lambda x: x.stem)
jpg_files_1 = sorted(jpg_files_1, key=lambda x: x.stem)

for txt, jpg in zip(txt_files_1, jpg_files_1):
    assert txt.stem == jpg.stem
    

In [64]:
len(txt_files_1), len(jpg_files_1), len(txt_files_0), len(jpg_files_0)

(649, 649, 637, 637)

In [65]:
indexes_1 = list(range(len(txt_files_1)))
indexes_0 = list(range(len(txt_files_0)))

random.shuffle(indexes_1)
random.shuffle(indexes_0)

def split_indexes(indexes):
    total_files = len(indexes)
    train_count = int(total_files * 0.7)
    test_count = int(total_files * 0.15)
    train_indexes = indexes[:train_count]
    #print(train_indexes)
    test_indexes = indexes[train_count: train_count + test_count]
    
    val_indexes = indexes[(train_count + test_count):]
    return train_indexes, test_indexes, val_indexes

def data_from_index(indexes, data):
    data_list = []
    for i in indexes:
        data_list.append(data[i])
    return data_list

train_files_i, test_files_i, val_files_i = split_indexes(indexes_0)

train_files = data_from_index(train_files_i, txt_files_0) + data_from_index(train_files_i, jpg_files_0)
test_files = data_from_index(test_files_i, txt_files_0) + data_from_index(test_files_i, jpg_files_0)
val_files = data_from_index(val_files_i, txt_files_0) + data_from_index(val_files_i, jpg_files_0)


train_files_1_i, test_files_1_i, val_files_1_i = split_indexes(indexes_1)

train_files_1 = data_from_index(train_files_1_i, txt_files_1) + data_from_index(train_files_1_i, jpg_files_1)
test_files_1 = data_from_index(test_files_1_i, txt_files_1) + data_from_index(test_files_1_i, jpg_files_1)
val_files_1 = data_from_index(val_files_1_i, txt_files_1) + data_from_index(val_files_1_i, jpg_files_1)

train_files.extend(train_files_1)
test_files.extend(test_files_1)
val_files.extend(val_files_1)




print(len(train_files), len(test_files), len(val_files))

1798 384 390


In [66]:
import shutil

def copy_files(files: list, dst: Union[str, Path]):
    dst_path = Path(dst)
    dst_path.mkdir(exist_ok=True)
    
    for file in files:
        shutil.copy(file, dst_path)
    
#source_folder = Path("/kaggle/working/for_data") 
train_folder = Path('/kaggle/working/train')
test_folder = Path('/kaggle/working/test')
valid_folder = Path('/kaggle/working/valid')

shutil.rmtree(train_folder)
shutil.rmtree(test_folder)
shutil.rmtree(valid_folder)


# subfolders = [f.path for f in os.scandir(images_data_folder) if f.is_dir()]
# print(subfolders)
#random.shuffle(subfolders)

copy_files(train_files, train_folder)
copy_files(test_files, test_folder)
copy_files(val_files, valid_folder)

def get_amount_of_files(dir: Path):
    files = [file for file in dir.glob("*")]
    return len(files)

print(get_amount_of_files(train_folder))
print(get_amount_of_files(test_folder))
print(get_amount_of_files(valid_folder))



1798
384
390


In [72]:
text = \
f"""train: {train_folder}
test: {test_folder}
val: {valid_folder}

# Classes 
names:
  0: without_silk_spot
  1: with_silk_spot

"""

print(text.encode())

Path('dataset.yaml').write_text(text)
# with open(f'dataset.yaml', 'w') as file:
#     file.write(f'train: /kaggle/working/train\nval: /kaggle/working/valid\ntest: /kaggle/working/test\n# Classes\nnames:\n  0: without_silk_spot\n  1: with_silk_spot\n')

b'train: /kaggle/working/train\ntest: /kaggle/working/test\nval: /kaggle/working/valid\n\n# Classes \nnames:\n  0: without_silk_spot\n  1: with_silk_spot\n\n'


146

In [73]:
folder_path = "/kaggle/working"
file_name = "dataset.yaml"

# Получение списка файлов в папке
files = os.listdir(folder_path)

# Поиск файла в списке
if file_name in files:
    print(f"Файл {file_name} найден в папке.")
else:
    print(f"Файл {file_name} не найден в папке.")

Файл dataset.yaml найден в папке.


In [74]:
from ultralytics import YOLO
model = YOLO('yolov8x.pt')

In [32]:


model.train(data='/kaggle/working/dataset.yaml', epochs = 100, patience = 20)

Ultralytics YOLOv8.1.43 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/dataset.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /kaggle/working/train.cache... 899 images, 0 backgrounds, 0 corrupt: 100%|██████████| 899/899 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/valid.cache... 195 images, 0 backgrounds, 0 corrupt: 100%|██████████| 195/195 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


100 epochs completed in 2.000 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 136.7MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68125494 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.34it/s]


                   all        195        284      0.677      0.569      0.626      0.265
     without_silk_spot        195        156      0.673      0.606      0.662      0.291
        with_silk_spot        195        128      0.681      0.533      0.589      0.239
Speed: 0.1ms preprocess, 13.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train2


lr/pg0,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▁▂▃▄▄▅▅▅▆▆▆▆▅▆▇▇▇▇▇▇▆▇▇█▇▇██▇▇▇▇▇██▇███
metrics/mAP50-95(B),▁▁▂▂▃▃▄▅▄▆▅▆▆▅▆▇▆▇▇▆▇▆▇▇▇▇▇▇▇▆▇▇▇▇██▇▇▇█
metrics/precision(B),▁▆▇█▃▅▅▅▅▅▅▅▆▅▆▆▇▆▆▅▆▆▆▆▆▇▆▇▆▆▆▇▆▇█▇▇▇▇▇
metrics/recall(B),▁▁▂▂▆▄▅▅▆▆▅▆▆▆▇█▆▇▇▇█▅█▇█▇███▇▇▇█▇██▇███
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▇█▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ae19c6b1b10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [40]:
#model.trainer.validator.dataloader.reset()
#тут произошла потеря inctance, модель почему-то не увидела разметку для некоторых картинок и отдала их в background
#но уже в следующей ячейке все хорошо
for p in range(20,50,10):
    model.train(data='/kaggle/working/dataset.yaml', epochs = 500, patience = p)

Ultralytics YOLOv8.1.44 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/dataset.yaml, epochs=500, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /kaggle/working/train.cache... 623 images, 276 backgrounds, 0 corrupt: 100%|██████████| 899/899 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/valid.cache... 23 images, 172 backgrounds, 0 corrupt: 100%|██████████| 195/195 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 500 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     27/500      15.4G      1.501      1.415      1.422          5        640: 100%|██████████| 57/57 [01:08<00:00,  1.21s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  2.21it/s]

                   all        195         47     0.0922      0.208     0.0504     0.0189



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     28/500      15.4G      1.339       1.24      1.327         40        640:  11%|█         | 6/57 [00:08<01:13,  1.43s/it]


KeyboardInterrupt: 

In [75]:
for p in range(30,50,10):
    model.train(data='/kaggle/working/dataset.yaml', epochs = 500, patience = p)

Ultralytics YOLOv8.1.44 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/dataset.yaml, epochs=500, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /kaggle/working/train... 899 images, 0 backgrounds, 0 corrupt: 100%|██████████| 899/899 [00:00<00:00, 968.66it/s]

train: New cache created: /kaggle/working/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/valid... 195 images, 0 backgrounds, 0 corrupt: 100%|██████████| 195/195 [00:00<00:00, 901.75it/s]

val: New cache created: /kaggle/working/valid.cache


...

                   all        195        280      0.628      0.567      0.578      0.244
EarlyStopping: Training stopped early as no improvement observed in last 40 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=40) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



41 epochs completed in 0.855 hours.
Optimizer stripped from runs/detect/train42/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train42/weights/best.pt, 136.7MB

Validating runs/detect/train42/weights/best.pt...
Ultralytics YOLOv8.1.44 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68125494 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.24it/s]


                   all        195        280      0.646      0.591       0.61      0.288
     without_silk_spot        195        145      0.669      0.752      0.726      0.366
        with_silk_spot        195        135      0.623       0.43      0.495      0.211
Speed: 0.1ms preprocess, 13.6ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train42


lr/pg0,▁▄██████████████████████████▇▇▇▇▇▇▇▇▇▇▇▇
lr/pg1,▁▄██████████████████████████▇▇▇▇▇▇▇▇▇▇▇▇
lr/pg2,▁▄██████████████████████████▇▇▇▇▇▇▇▇▇▇▇▇
metrics/mAP50(B),█▆▇▇▇▇▇▁▅█▇▇▇▇▇▇▇██▇▇▇█▇█▇█▇▇▇▇█▇▇█▇▇█▇█
metrics/mAP50-95(B),█▆▇▇▇▇▇▁▅▇▇▇▆▆▇▇▆▇█▇▇▇█▆▇▇▇▇▇▇▇█▇▇▇▇▇▇▇█
metrics/precision(B),▅▁▁▃█▆▄▃▅▅▃▆▄▅▃▃█▄▆▄▇▆▄▃▅▄▄▅▃▅▄▄▄▁▅▁▅▅▃▅
metrics/recall(B),▇▅▇▇▅▆▇▁▄▇▇▇▆▆▇▇▄▇▇▇▆▇▇▆▇▇▇▇▇▆▆█▇█▇▇▇▇█▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▄▄▆█▇▆▇█▇▇▇▇▇█▇▆█▄▅▅▆▆▅▄▅▅▅▃▄▅▄▄▅▃▂▁▁▂▃▁


In [76]:
model.train(data='/kaggle/working/dataset.yaml', epochs = 500, patience = 20)

Ultralytics YOLOv8.1.44 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/dataset.yaml, epochs=500, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train422, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/working/train.cache... 899 images, 0 backgrounds, 0 corrupt: 100%|██████████| 899/899 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/valid.cache... 195 images, 0 backgrounds, 0 corrupt: 100%|██████████| 195/195 [00:00<?, ?it/s]


...


21 epochs completed in 0.444 hours.
Optimizer stripped from runs/detect/train422/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train422/weights/best.pt, 136.7MB

Validating runs/detect/train422/weights/best.pt...
Ultralytics YOLOv8.1.44 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68125494 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.27it/s]


                   all        195        280      0.666      0.601      0.602      0.283
     without_silk_spot        195        145      0.654      0.772      0.717      0.365
        with_silk_spot        195        135      0.677       0.43      0.486      0.201
Speed: 0.1ms preprocess, 13.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train422


lr/pg0,▁▄███████████████████
lr/pg1,▁▄███████████████████
lr/pg2,▁▄███████████████████
metrics/mAP50(B),█▇▁█▅▆▆▇▇█▇▆▇▅▇▇▇▇▇▇█
metrics/mAP50-95(B),█▆▁▇▄▆▆▆▆▇▇▅▆▃▇▆▇▆▆▆█
metrics/precision(B),▆▄▆█▃▂▄▂▄▆▅▅▄▁▅▄▄▅▄▅▆
metrics/recall(B),█▇▁▆▅▇▆▇▇▇▇▇▇▅▇▇█▇█▇█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▁▅▆█▇▇▇▇█▇▇▇▇█▇▆▇▅▆▅▇


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7837e40e72e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804